In [ ]:
#Proyecto 2 de Gómez Guzmán Luis Enrique

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split, cross_val_score, GridSearchCV)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_recall_fscore_support,
    roc_curve,
    roc_auc_score
)
import os

class DataLoadError(Exception):
    """Excepción para errores de carga de datos."""

    def __init__(self, mensaje = "Error al cargar los datos"):
        self.mensaje = mensaje
        super().__init__(self.mensaje)

class ModelTrainingError(Exception):
    """Excepción para errores
    durante el entrenamiento de modelos."""

    def __init__(self, mensaje = "Error durante el entrenamiento del modelo"):
        self.mensaje = mensaje
        super().__init__(self.mensaje)

class Clasificadores():
    """
    Clase para gestionar los clasificadores.
    Se implementan diversos métodos de clasificación comparatia
    """
    def __init__(self, X_train, y_train):
        """
        Inicializa los clasificadors con datos de entrenamiento.

        """

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)

        self.X_train = X_train_scaled
        self.y_train = y_train

        # Inicializar clasificadores con búsqueda de hiperparámetros

        self.modelos = {
            'Regresión Logística': {
                'modelo': LogisticRegression(random_state = 42),
                'params': {
                    'C': [0.001, 0.01, 0.1, 1, 10, 100],
                    'penalty': ['l1', 'l2'],
                    'solver': ['liblinear']
                }
            },
            'KNN': {
                'modelo': KNeighborsClassifier(),
                'params': {
                    'n_neighbors': [3, 5, 7, 9, 11],
                    'weights': ['uniform', 'distance'],
                    'algorithm': ['auto', 'ball_tree', 'kd_tree']
                }
            },
            'SVM': {
                'modelo': SVC(probability = True, random_state = 42),
                'params': {
                    'C': [0.1, 1, 10, 100],
                    'kernel': ['linear', 'rbf', 'poly'],
                    'gamma': ['scale', 'auto']
                }
            },
            'Árbol de Decisión': {
                'modelo': DecisionTreeClassifier(random_state = 42),
                'params': {
                    'max_depth': [3, 5, 7, 10],
                    'min_samples_split': [2, 5, 10],
                    'criterion': ['gini', 'entropy']
                }
            },
            'Random Forest': {
                'modelo': RandomForestClassifier(random_state = 42),
                'params': {
                    'n_estimators': [50, 100, 200],
                    'max_depth': [3, 5, 7, 10],
                    'min_samples_split': [2, 5, 10]
                }
            }
        }

        self.resultados = {}
        self.mejores_modelos = {}

    def entrenar_modelos(self):
        """
        Entrena los modelos con búsqueda de hiperparámetros.

        """
        try:
            for nombre, config in self.modelos.items():

                # Realizar búsqueda de cuadrícula de hiperparámetros

                grid_search = GridSearchCV(
                    estimator = config['modelo'],
                    param_grid = config['params'],
                    cv = 5,
                    scoring = 'accuracy',
                    n_jobs = -1
                )

                # Ajustar el modelo con búsqueda de hiperparámetros

                grid_search.fit(self.X_train, self.y_train)

                # Guardar el mejor modelo

                self.mejores_modelos[nombre] = grid_search.best_estimator_
                print(f"\nMejores parámetros para {nombre}:")
                print(grid_search.best_params_)
        except Exception as e:
            raise ModelTrainingError(f"Error entrenando modelos: {str(e)}")

    def evaluar_modelos(self, X_test, y_test):
        """
        Evalúa el rendimiento de los modelos.

        """
        X_test_scaled = self.scaler.transform(X_test)

        for nombre, modelo in self.mejores_modelos.items():
            y_pred = modelo.predict(X_test_scaled)

            # Métricas adicionales

            precision, recall, f1, _ = precision_recall_fscore_support(
                y_test, y_pred, average = 'weighted'
            )

            # Calcular curva ROC

            y_pred_proba = modelo.predict_proba(X_test_scaled)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_auc = roc_auc_score(y_test, y_pred_proba)

            self.resultados[nombre] = {
                'accuracy': accuracy_score(y_test, y_pred),
                'f1_score': f1_score(y_test, y_pred, average='weighted'),
                'precision': precision,
                'recall': recall,
                'matriz_confusion': confusion_matrix(y_test, y_pred),
                'reporte_clasificacion': classification_report(y_test, y_pred),
                'fpr': fpr,
                'tpr': tpr,
                'roc_auc': roc_auc
            }

    def validacion_cruzada(self, X, y, cv = 5):
        """
        Realiza validación cruzada para cada modelo.

        """
        X_scaled = self.scaler.transform(X)

        print("\n--- VALIDACIÓN CRUZADA ---")
        for nombre, modelo in self.mejores_modelos.items():

            scores = cross_val_score(modelo, X_scaled, y, cv = cv,
                                     scoring = 'accuracy')

            print(f"\n{nombre}:")
            print(f"Precisión (CV):{scores.mean():.2%} \
            (+/- {scores.std() * 2:.2%}))")

    def visualizar_resultados(self):
        """
        Genera visualizaciones comparativas de los modelos.

        """
        # Crear directorio para guardar gráficas si no existe

        os.makedirs('graficas', exist_ok = True)

        # Grafica de precision

        plt.figure(figsize = (12, 6))
        accuracies = [res['accuracy'] for res in self.resultados.values()]
        nombres = list(self.resultados.keys())

        bars = plt.bar(nombres, accuracies)
        plt.title('Precisión de Modelos de Clasificación', fontsize = 15)
        plt.xlabel('Modelo', fontsize = 12)
        plt.ylabel('Accuracy', fontsize = 12)
        plt.xticks(rotation = 45)

        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.2%}',
                     ha = 'center', va = 'bottom')

        plt.tight_layout()
        plt.savefig('graficas/comparacion_accuracies.png')
        plt.close()

        # Curvas ROC

        plt.figure(figsize = (10, 8))
        for nombre, resultado in self.resultados.items():
            plt.plot(
                resultado['fpr'],
                resultado['tpr'],
                label = f'{nombre} (AUC = {resultado["roc_auc"]:.2f})'
            )

        plt.plot([0, 1], [0, 1], linestyle='--', label = 'Línea base')
        plt.title('Curvas ROC de Clasificadores', fontsize = 15)
        plt.xlabel('Tasa de Falsos Positivos', fontsize = 12)
        plt.ylabel('Tasa de Verdaderos Positivos', fontsize = 12)
        plt.legend(loc = 'lower right')
        plt.savefig('graficas/curvas_roc.png')
        plt.close()

        # Matrices de confusion

        for nombre, resultado in self.resultados.items():
            plt.figure(figsize = (8, 6))
            sns.heatmap(
                resultado['matriz_confusion'],
                annot = True,
                fmt = 'd',
                cmap = 'Blues'
            )
            plt.title(f'Matriz de Confusión - {nombre}')
            plt.xlabel('Predicción')
            plt.ylabel('Real')
            plt.tight_layout()
            plt.savefig(f'graficas/matriz_confusion_{nombre}.png')
            plt.close()

        #Impreson dr  resultadis

        print("\n--- COMPARACIÓN DE MODELOS ---")
        for nombre, resultado in self.resultados.items():
            print(f"\n{nombre}:")
            print(f"Accuracy: {resultado['accuracy']:.2%}")
            print(f"F1 Score: {resultado['f1_score']:.2%}")
            print(f"Precisión: {resultado['precision']:.2%}")
            print(f"Recall: {resultado['recall']:.2%}")
            print(f"AUC-ROC: {resultado['roc_auc']:.2f}")
            print("\nReporte de Clasificación:")
            print(resultado['reporte_clasificacion'])

def preparar_datos(ruta_archivo):
    """
    Prepara los datos para clasificación.

    """
    try:
        # Cargar datos desde CSV

        datos = pd.read_csv('/content/drive/MyDrive/cancer.csv')

        # Información del dataset

        print("Información del Dataset:")
        print(datos.info())


        le = LabelEncoder()

        # Identificar columna objetivo

        if 'diagnosis' in datos.columns:
            y = le.fit_transform(datos['diagnosis'])
            X = datos.select_dtypes \
             (include=[np.number]).drop(columns=['diagnosis'])
        else:
            # Usar primera columna categórica como etiqueta

            columna_objetivo = datos.select_dtypes \
             (include = ['object']).columns[0]
            y = le.fit_transform(datos[columna_objetivo])
            X = datos.select_dtypes \
             (include = [np.number]).drop(columns = [columna_objetivo])

        # Eliminar columnas con alta correlación

        corr_matrix = X.corr().abs()
        upper_tri = corr_matrix.where \
         (np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool))
        to_drop = [column for column in upper_tri.columns if \
                   any(upper_tri[column] > 0.95)]
        X = X.drop(columns = to_drop)

        X, y = preparar_datos('/content/drive/MyDrive/cancer.csv')
        return X, y

    except Exception as e:
        raise DataLoadError(f"Error preparando datos: {e}")

# Implementacion del metodo main

def main():
    """
    Función principal que ejecuta el proyecto de clasificación.
    """
    try:
        # Intentar cargar datos de diferentes rutas posibles

        rutas_drive = [
            'cancer.csv',
            './cancer.csv',
            '../cancer.csv',
            './data/cancer.csv',
            '/content/cancer.csv',
            '/content/drive/MyDrive/cancer.csv',
        ]

        ruta_archivo = '/content/drive/MyDrive/cancer.csv'

        for ruta in rutas_drive:
            try:
                if os.path.exists(ruta):
                    ruta_archivo = ruta
                    break
            except Exception:
                continue

        if not ruta_archivo:
            raise FileNotFoundError("No se encontró el archivo cancer.csv")

        # Preparar datos

        ruta_archivo = '/content/drive/MyDrive/cancer.csv'
        X, y = preparar_datos(ruta_archivo)

        # Información sobre distribución de clases

        print("\nDistribución de clases:")
        unique, counts = np.unique(y, return_counts=True)
        for clase, cantidad in zip(unique, counts):
            print(f"Clase {clase}: {cantidad} instancias")

        # División de datos

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        # Creacion de la  instancia de clasificadores

        clasificadores = Clasificadores(X_train, y_train)

        # Entrenar modelos con búsquda de hiperparámetros

        clasificadores.entrenar_modelos()

        # Evaluar modelos

        clasificadores.evaluar_modelos(X_test, y_test)

        # Realizar validación cruzada

        clasificadores.cross_validation(X_train, y_train)

        # Visualizar y comparar resultados

        clasificadores.visualizar_resultados()

    except FileNotFoundError as e:
        print(f"Error: {e}. Por favor, asegúrate de tener un archivo \
        'cancer.csv' en el directorio correcto.")
    except DataLoadError as e:
        print(f"Error de carga de datos: {e}")
    except ModelTrainingError as e:
        print(f"Error de entrenamiento: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")


if __name__ == "__main__":
    main()

Mounted at /content/drive
Información del Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       499 non-null    int64  
 1   diagnosis                499 non-null    object 
 2   radius_mean              499 non-null    float64
 3   texture_mean             499 non-null    float64
 4   perimeter_mean           499 non-null    float64
 5   area_mean                499 non-null    float64
 6   smoothness_mean          499 non-null    float64
 7   compactness_mean         499 non-null    float64
 8   concavity_mean           499 non-null    float64
 9   concave points_mean      499 non-null    float64
 10  symmetry_mean            499 non-null    float64
 11  fractal_dimension_mean   499 non-null    float64
 12  radius_se                499 non-null    float64
 13  texture_se               499 